In [9]:
import pandas as pd
import os

pd.set_option('display.max_columns', 500)

data_path = os.getcwd()[:-len('notebooks')] + 'Data/'
db_credit_card_fraud_detection_small = data_path + 'Credit_card_fraud_detection_small/'
db_beijing_path = data_path + 'Beijing_winter_Olympic_Games/'
transactional_data = data_path + 'Transactional_data/'

In [6]:
os.listdir(transactional_data)

['.DS_Store',
 'goat of tennis.csv',
 'Highest Holywood Grossing Movies.csv',
 'trains vs planes.csv',
 'Loan_Default.csv',
 'New York Citibike Trips.csv',
 'Business Sales Transaction.csv']

# Beijing

In [18]:
os.listdir(db_beijing_path)

['hockey_results.csv',
 'curling_results.csv',
 'athletes.csv',
 'technical_officials.csv',
 'medals_total.csv',
 'coaches.csv',
 'entries_discipline.csv',
 'hockey_players_stats.csv',
 'medals.csv',
 'events.csv']

# Ejercicio Práctica sábado:

Como comité olímpico, nos solicitan analizar la relación que existe entre el número de participantes enviados por cada país y el número de medallas obtenidos en los tres niveles. 

Nos interesa analizar este proceso a nivel país, y nos gustaría poder revisar la cantidad de atletas enviados por cada país (cantidad total, cantidad de mujeres y cantidad de hombres), total de medallas obtenidas, total de medallas de oro, total de medallas de plata y total de medallas de bronce (segmentados por hombres y mujeres). Además de esto, nos interesa contar la cantidad de coaches que envió cada país, de igual forma, divido por mujeres y hombres. Y por último agregar las columnas para los referies, la intención es analizar si la cantidad de personas enviadas por cada país está relacionada con la cantidad de medallas traidas por cada país. 

# Ecommerce

In [16]:
sales_ecommerce = pd.read_csv(transactional_data + 'Business Sales Transaction.csv', low_memory = False)
sales_ecommerce.head()

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,536365,2018-12-01,85123A,Cream Hanging Heart T-Light Holder,1.88,6,17850.0,United Kingdom
1,536365,2018-12-01,71053,White Moroccan Metal Lantern,2.01,6,17850.0,United Kingdom
2,536365,2018-12-01,84406B,Cream Cupid Hearts Coat Hanger,1.91,8,17850.0,United Kingdom
3,536365,2018-12-01,84029G,Knitted Union Flag Hot Water Bottle,2.01,6,17850.0,United Kingdom
4,536365,2018-12-01,84029E,Red Woolly Hottie White Heart,2.01,6,17850.0,United Kingdom


# Ejercicio:

Nos solicitan revisar si es posible establecer una relación entre el país y la demanda de ciertos productos. La idea es que conociendo el país se pueda predecir las cantidades de los top tres productos para cada país (su precio, la cantidad de unidades vendidas) esto agrupado por mes. 

* 1. Unidad muestral: País.
* 2. Quantity variable auxiliar para el cálculo de la demanda.
* 3. Segunda variable auxiliar precio.
* 4. Esto para los 3 productos más vendidos por cada país. 
* 5. Agrupación a nivel mes.

In [19]:
sales_ecommerce.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536350 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionNo  536350 non-null  object 
 1   Date           536350 non-null  object 
 2   ProductNo      536350 non-null  object 
 3   ProductName    536350 non-null  object 
 4   Price          536350 non-null  float64
 5   Quantity       536350 non-null  int64  
 6   CustomerNo     536295 non-null  float64
 7   Country        536350 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.7+ MB


In [25]:
sales_ecommerce['Date'][0][8:]

'01'

In [21]:
# Checar si todos son del mismo año
sales_ecommerce['Date'].value_counts(1, dropna = False)

2019-12-05    0.009880
2019-12-08    0.009179
2019-11-29    0.007999
2019-11-16    0.007749
2019-11-11    0.007568
                ...   
2019-03-13    0.000996
2018-12-19    0.000970
2019-05-01    0.000839
2018-12-22    0.000539
2019-02-06    0.000520
Name: Date, Length: 305, dtype: float64

In [26]:
sales_ecommerce['Date'].value_counts(1)

2019-12-05    0.009880
2019-12-08    0.009179
2019-11-29    0.007999
2019-11-16    0.007749
2019-11-11    0.007568
                ...   
2019-03-13    0.000996
2018-12-19    0.000970
2019-05-01    0.000839
2018-12-22    0.000539
2019-02-06    0.000520
Name: Date, Length: 305, dtype: float64

In [40]:
sales_ecommerce['Date'][0][:7]

'2018-12'

In [38]:
' 2018-12-23 '.strip()[:4]

'2018'

In [39]:
' 2018-12-23 '.strip().split('-')

['2018', '12', '23']

In [42]:
df = sales_ecommerce.copy()

In [45]:
df['year'] = df['Date'].map(lambda x:x.strip()[:4])
df['month'] = df['Date'].map(lambda x:x.strip()[5:7])
df['day'] = df['Date'].map(lambda x:x.strip()[8:])
df['year-month'] = df['Date'].map(lambda x:x.strip()[:7])

df['year'].value_counts(1, dropna = False)

2019    0.921518
2018    0.078482
Name: year, dtype: float64

In [47]:
df.head(1)

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country,year,month,day,year-month
0,536365,2018-12-01,85123A,Cream Hanging Heart T-Light Holder,1.88,6,17850.0,United Kingdom,2018,12,01,2018-12


In [51]:
df.loc[df['Country'] == 'United Kingdom'][['Quantity', 'ProductNo']].groupby('ProductNo').agg(['sum']).sort_values(by = ('Quantity', 'sum'), ascending = False)

,Quantity
,sum
ProductNo,
22197,52821
84077,48504
85099B,43107
84879,33569
85123A,33227
...,...
79323P,-4
79323W,-8


In [ ]:
df[['Country', 'Quantity', 'ProductNo']]

In [52]:
# Opciones:

# 1. For
# 2. GroupBy
# 3. Creación de llave

In [64]:
# 1. For
%time
pa = list(set(df['Country']))
mo = list(set(df['year-month']))
products = dict()

for p in pa:
    aux = df.loc[df['Country'] == p].copy()
    for m in mo:
        aux1 = aux.loc[df['year-month'] == m].copy()
        aux2 = aux1[['Quantity', 'ProductNo']].groupby('ProductNo').agg(['sum']).sort_values(by = ('Quantity', 'sum'), ascending = False)[:3]
        products[p  + '_' + m] = aux2

products

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 10 µs


{'Finland_2019-05': Empty DataFrame
 Columns: [(Quantity, sum)]
 Index: [],
 'Finland_2019-10':           Quantity
                sum
 ProductNo         
 84997D          76
 21212           48
 23307           48,
 'Finland_2019-01':           Quantity
                sum
 ProductNo         
 21967          144
 21980          144
 21983          144,
 'Finland_2019-11':           Quantity
                sum
 ProductNo         
 84997D         144
 23084           96
 22598           72,
 'Finland_2019-02':           Quantity
                sum
 ProductNo         
 21212           24
 20677           16
 21238           16,
 'Finland_2019-04':           Quantity
                sum
 ProductNo         
 23078           96
 23077           80
 23076           72,
 'Finland_2019-09':           Quantity
                sum
 ProductNo         
 23077          120
 23078           96
 84997D          44,
 'Finland_2018-12':           Quantity
                sum
 ProductNo         
 2121

In [68]:
# 2. GroupBy
%time
aux3 = df[['Country', 'year-month', 'ProductNo', 'Quantity']].groupby(['Country', 'year-month', 'ProductNo']).agg('sum')
aux3.sort_values(by = 'Country').sort_values(by = 'year-month').sort_values(by = 'Quantity', ascending = False)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


Quantity
Country        year-month ProductNo          
United Kingdom 2019-11    84826         12551
                          22197         11728
               2019-04    84077          9471
               2019-11    23084          8335
               2019-10    84077          8084
...                                       ...
               2019-07    21439          -373
               2019-08    21877          -598
Japan          2019-04    22328          -624
United Kingdom 2019-04    47566B         -950
               2018-12    84347         -7078

[70902 rows x 1 columns]

In [76]:
# 3. key
%time
df['key'] = df['Country'] + '_' + df['year-month'] + '_' + df['ProductNo']
aux4 = df[['key', 'Quantity']].groupby('key').sum().sort_values(by = ('Quantity'), ascending = False)


CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 11.9 µs


In [74]:
dffinal = df[['Country', 'day']].groupby('Country').count()
dffinal.columns = ['Número de transcacciones por país']
dffinal

,Número de transcacciones por país
Country,
Australia,1704
Austria,887
Bahrain,17
Belgium,2539
Brazil,31
Canada,150
Channel Islands,629
Cyprus,582
Czech Republic,28


In [78]:
aux4.index.astype(str)

Index(['United Kingdom_2019-11_84826', 'United Kingdom_2019-11_22197',
       'United Kingdom_2019-04_84077', 'United Kingdom_2019-11_23084',
       'United Kingdom_2019-10_84077', 'United Kingdom_2019-11_22086',
       'United Kingdom_2019-05_22197', 'United Kingdom_2019-08_84879',
       'United Kingdom_2019-12_22197', 'United Kingdom_2019-11_85099B',
       ...
       'United Kingdom_2019-07_23001', 'United Kingdom_2019-12_46000M',
       'United Kingdom_2019-12_22313', 'United Kingdom_2019-12_46000S',
       'United Kingdom_2019-09_18097C', 'United Kingdom_2019-07_21439',
       'United Kingdom_2019-08_21877', 'Japan_2019-04_22328',
       'United Kingdom_2019-04_47566B', 'United Kingdom_2018-12_84347'],
      dtype='object', name='key', length=70902)

In [82]:
aux4['country'] = aux4.index.astype(str).map(lambda x:x.strip().split('_')[0])
aux4['year-month'] = aux4.index.astype(str).map(lambda x:x.strip().split('_')[1])
aux4['prod'] = aux4.index.astype(str).map(lambda x:x.strip().split('_')[2])

In [86]:
columns = list(set(aux4['year-month']))
Country = list(set(aux4['country']))
results = dict()
for p in Country[:2]:
    aux5 = aux4.loc[aux4['country'] == p]
    for c in columns[:2]:
        aux6 = aux5.loc[aux5['year-month'] == c].sort_values(by = 'Quantity', ascending = False)
        result = aux6[['Quantity', 'prod']].head(3)
        results[p + '_' + c] = result

In [87]:
results

{'Finland_2019-05': Empty DataFrame
 Columns: [Quantity, prod]
 Index: [],
 'Finland_2019-01':                        Quantity   prod
 key                                   
 Finland_2019-01_21980       144  21980
 Finland_2019-01_21983       144  21983
 Finland_2019-01_21985       144  21985,
 'USA_2019-05': Empty DataFrame
 Columns: [Quantity, prod]
 Index: [],
 'USA_2019-01': Empty DataFrame
 Columns: [Quantity, prod]
 Index: []}